# Move the dataset to VM.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Create directories

In [ ]:
mkdir train_data

In [ ]:
cd train_data

/content/train_data


In [ ]:
mkdir lateral

mkdir: cannot create directory ‘lateral’: File exists


In [ ]:
mkdir frontal

mkdir: cannot create directory ‘frontal’: File exists


In [ ]:
ls

In [ ]:
mkdir test_data

In [ ]:
import csv
from shutil import copyfile
import os
import zipfile

In [ ]:
# copy all images in train folder
csvReader = csv.DictReader(open("/content/drive/My Drive/Sirius challenge/train_new.csv"))
for row in csvReader:
    if row['label'] == 'frontal':
      dst = '/content/train_data/frontal/' + row['target_name']
    elif row['label'] == 'lateral':
      dst = '/content/train_data/lateral/' + row['target_name']
    copyfile('/content/drive/My Drive/Sirius challenge/chexpertData_new/' + row['target_name'], dst)

In [ ]:
# copy all images in test folder
csvReader = csv.DictReader(open("/content/drive/My Drive/Sirius challenge/test_new.csv"))
for row in csvReader:
    dst = '/content/test_data/' + row['target_name']
    copyfile('/content/drive/My Drive/Sirius challenge/chexpertData_new/' + row['target_name'], dst)

### Zip

In [ ]:
# zip both folders
# then we can upload them in google drive and do not repeat all these actions

def zipdir(path, ziph):
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

zipf = zipfile.ZipFile('/content/train_data.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('/content/train_data', zipf)
zipf.close()

In [ ]:
zipf = zipfile.ZipFile('/content/test_data.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('/content/test_data', zipf)
zipf.close()

In [ ]:
zipf = zipfile.ZipFile('/content/GB7.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('/content/drive/My Drive/Sirius challenge/GB7_new', zipf)
zipf.close()

### Unzip


In [ ]:
# unzip GB7 dataset
with zipfile.ZipFile('/content/drive/My Drive/GB7.zip', 'r') as zip_ref:
    zip_ref.extractall('')

In [ ]:
# unzip train dataset
with zipfile.ZipFile('/content/drive/My Drive/train_data.zip', 'r') as zip_ref:
    zip_ref.extractall('')

In [ ]:
# unzip test dataset
with zipfile.ZipFile('/content/drive/My Drive/test_data.zip', 'r') as zip_ref:
    zip_ref.extractall('')

# Main code


In [ ]:
import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import cv2
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import csv
from PIL import Image
import numpy 
from shutil import copyfile
import random

### Increase training images 


In [ ]:
!pip install albumentations

     |████████████████████████████████| 634kB 4.6MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654020 sha256=78f05aa96f6af757d3401c947cbe04db46f7a002b8f596c432fb2adc6d4dcdb3
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [ ]:
import albumentations as A

augmentation = A.Compose([
        A.RandomRotate90(p=0.4),
        A.Flip(p=0.4),
        A.OneOf([
            A.IAAAdditiveGaussianNoise(),
            A.GaussNoise(),
        ], p=0.4),
    ], p=1)

In [ ]:
for root, dirs, files in os.walk('/content/train_data/', topdown=True):
    for name in files:
      if random.random() > 0.99:
        img = cv2.imread(os.path.abspath(os.path.join(root, name)),0)
        new = augmentation(image = img)['image']
        components = os.path.abspath(os.path.join(root, 'a_'+name)).split('/')
        path = os.path.abspath(os.path.join(root, components[-1]))
        cv2.imwrite(path, new)

### Set hyperparameters

In [ ]:
batch_size = 64
test_size = 0
valid_size = 0.1

transform = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

In [ ]:
data = datasets.ImageFolder('/content/train_data', transform=transform)
data

Dataset ImageFolder
    Number of datapoints: 3199
    Root location: /content/train_data
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [ ]:
num_data = len(data)
indices_data = list(range(num_data))
np.random.shuffle(indices_data)
split_tt = int(np.floor(test_size * num_data))
train_idx, test_idx = indices_data[split_tt:], indices_data[:split_tt]

num_train = len(train_idx)
indices_train = list(range(num_train))
np.random.shuffle(indices_train)
split_tv = int(np.floor(valid_size * num_train))
train_new_idx, valid_idx = indices_train[split_tv:],indices_train[:split_tv]

train_sampler = SubsetRandomSampler(train_new_idx)
test_sampler = SubsetRandomSampler(test_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [ ]:
train_loader = torch.utils.data.DataLoader(data, batch_size=batch_size,
    sampler=train_sampler, num_workers=1)
valid_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=1)
test_loader = torch.utils.data.DataLoader(data, sampler = test_sampler, batch_size=batch_size, 
    num_workers=1)
classes = [1,0]

In [ ]:
len(test_loader)*batch_size + len(valid_loader)*batch_size + len(train_loader)*batch_size

3200

In [ ]:
# define neural net architecture
train_on_gpu = torch.cuda.is_available()

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.conv2 = nn.Conv2d(16, 64, 3)
        self.fc1 = nn.Linear(64*14*14, 512)
        self.fc2 = nn.Linear(512, 80)
        self.fc3 = nn.Linear(80, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.2)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(self.conv1(x))
        x = self.pool(F.elu(self.conv2(x)))
        x = x.view(-1, 64*14*14)
        x = F.elu(self.fc1(x))
        x = F.elu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.sig(x)
        return x

model = Net()
print(model)

if train_on_gpu:
    model.cuda()

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=12544, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=80, bias=True)
  (fc3): Linear(in_features=80, out_features=1, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.2, inplace=False)
  (sig): Sigmoid()
)


In [ ]:
# set loss function and optimization algorithm
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# training loop
n_epochs = 7 
valid_loss_min = np.Inf 
t2vRatio = 1.9           
t2vEpochs = 2   

for epoch in range(1, n_epochs+1):
    train_loss = 0.0
    valid_loss = 0.0
    model.train()
    for data, target in train_loader:
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data).squeeze(1)
        # print(output.cpu(), target.cpu())
        loss = criterion(output, target.type_as(output))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    model.eval()
    for data, target in valid_loader:
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        output = model(data).squeeze(1)
        loss = criterion(output, target.type_as(output))
        valid_loss += loss.item()*data.size(0)
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        torch.save(model.state_dict(), 'challenge.pth')
        valid_loss_min = valid_loss
    if valid_loss > t2vRatio * valid_loss:
      t2vEpochs -= 1                
      if t2vEpochs < 1:
        print("Validation loss too high; halting to prevent overfitting")
        break

Epoch: 1 	Training Loss: 0.102551 	Validation Loss: 0.001761
Validation loss decreased (inf --> 0.001761).  Saving model ...
Epoch: 2 	Training Loss: 0.032721 	Validation Loss: 0.001128
Validation loss decreased (0.001761 --> 0.001128).  Saving model ...
Epoch: 3 	Training Loss: 0.015349 	Validation Loss: 0.000196
Validation loss decreased (0.001128 --> 0.000196).  Saving model ...
Epoch: 4 	Training Loss: 0.007456 	Validation Loss: 0.000401
Epoch: 5 	Training Loss: 0.004847 	Validation Loss: 0.000064
Validation loss decreased (0.000196 --> 0.000064).  Saving model ...
Epoch: 6 	Training Loss: 0.013878 	Validation Loss: 0.000586
Epoch: 7 	Training Loss: 0.001417 	Validation Loss: 0.000010
Validation loss decreased (0.000064 --> 0.000010).  Saving model ...


In [ ]:
# create file predictions.csv to store the predictions
copyfile('/content/drive/My Drive/sample_subm_1.csv', '/content/result.csv')
# evaluate trained model
model.eval()
correct = 0
with open('/content/drive/My Drive/sample_subm_1.csv','r') as csvinput:
    # we use the random.csv template
    with open('/content/result.csv', 'w') as csvoutput:
        writer = csv.writer(csvoutput)
        for row in csv.reader(csvinput):
            if row[0] != 'target_name':
                if row[0][:7] == 'patient':
                  # the body of the table
                  img = cv2.imread('/content/test_data/'+row[0],0)
                else:
                  img = cv2.imread('/content/drive/My Drive/Sirius challenge/GB7_new/'+row[0],0)
                  img = np.invert(img)
                
                
                if len(np.shape(img)) == 2:
                  img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
                # use the same transformation as it was in training
                transform = transforms.Compose([
                  transforms.Resize(size=(64, 64)),
                  transforms.ToTensor(),
                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                ])
                img = transform(Image.fromarray(img)).cuda()
                with torch.no_grad() as tn:
                  # get the prediction
                  output = model(img[None, ...])
                  # set number representation
                  res = '{:.6f}'.format(1-output.cpu().numpy()[0][0])
                  writer.writerow([row[0]]+[res])
            
                # just to observe the improvement in the classification after code changes
                if float(res) > 0.5 and row[0][-9:] == 'view1.png':
                  correct += 1
                elif float(res) <= 0.5 and (row[0][-9:] == 'view2.png' or row[0][-9:] == 'view3.png'):
                  correct += 1
                # the following images are incorrectly named so we will process them separately
                elif row[0] == 'patient05974study4view1.png' and float(res) <= 0.5:
                  correct +=1
                elif row[0] == 'patient00942study4view2.png' and float(res) > 0.5:
                  correct +=1
                elif row[0] == 'patient01645study8view2.png' and float(res) > 0.5:
                  correct +=1
                elif row[0] == 'patient00204study7view2.png' and float(res) > 0.5:
                  correct +=1
                elif row[0] == 'patient01489study15view2.png' and float(res) > 0.5:
                  correct +=1
                elif row[0] == 'patient03368study1view2.png' and float(res) > 0.5:
                  correct +=1
                # prediction errors
                elif row[0][:7] == 'patient': 
                  print(row[0], ' ', res)
            else:
                # the header of the table
                writer.writerow([row[0]]+[row[1]]) 
print("Final accuracy {:-4}/800".format(correct))

Final accuracy  800/800


In [ ]:
torch.save(model.state_dict(), 'challenge.pth')